In [77]:
%pip install -qU pypdf 
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



In [78]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
file_path = "./solving_sudoku.pdf"
loader = PyPDFLoader(file_path)

load_dotenv()
docs = loader.load()

In [79]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [40]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

In [56]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for solving sudoku."
    "You will be provided with the current state of the puzzle as well as the solution of the puzzle." 
    "Choose a random cell that you think can be deduced from the current state of the puzzle and decide which number belongs there."
    #"Verify that the number you deduced is correct, using the solution. "
    "Explain the logic that led you to that number, without referencing the solution."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:
from sudoku import Sudoku

In [76]:
new_sudoku = Sudoku(3).difficulty(0.9)
new_sudoku.board[8][4] = 7;
new_sudoku.show()

Puzzle has multiple solutions
+-------+-------+-------+
|       |   1   |       |
|       |       |       |
| 3     |       |       |
+-------+-------+-------+
|       |       |       |
|       |       |       |
|   8   |       |       |
+-------+-------+-------+
|     6 |   2   |       |
|       |     6 |   3   |
|   1   |   7 8 |       |
+-------+-------+-------+



In [64]:
results = rag_chain.invoke({"input": str(new_sudoku.board)+str(new_sudoku.solve().board)})
print(results['answer'])

Let's examine the puzzle and see if we can determine the value for a specific cell using logic.

### Initial Observations:
- The given puzzle is a 9x9 Sudoku grid with some cells already filled.
- We have the following initial numbers placed in the grid:
  - Row 1: 1 at column 5
  - Row 3: 3 at column 1
  - Row 6: 8 at column 2
  - Row 7: 6 at column 3, and 2 at column 5
  - Row 8: 6 at column 6, and 3 at column 8
  - Row 9: 1 at column 2, and 8 at column 6

### Step-by-Step Deduction:
Let's focus on a particular box to deduce a number. Consider the bottom middle 3x3 box (rows 7 to 9, columns 4 to 6). We know:
- Row 7 has a 6 at column 3 and a 2 at column 5.
- Row 8 has a 6 at column 6.
- Row 9 has an 8 at column 6.

Let's focus on cell (9, 5):
- The numbers available to place in this box (based on the given values in this box) are: 2, 4, 5, 7, and 9.
- However, we have a 2 in row 7, column 5, so 2 cannot be placed in cell (9, 5).
- The number 8 is already in row 9, and 6 in row 8, so 

In [71]:
solution = new_sudoku.solve()
solution.show()

Puzzle has exactly one solution
+-------+-------+-------+
| 2 6 4 | 8 1 3 | 7 9 5 |
| 1 7 5 | 9 6 4 | 3 2 8 |
| 3 9 8 | 2 5 7 | 4 6 1 |
+-------+-------+-------+
| 7 4 2 | 6 9 1 | 5 8 3 |
| 6 5 3 | 7 8 2 | 9 1 4 |
| 9 8 1 | 4 3 5 | 6 7 2 |
+-------+-------+-------+
| 8 3 6 | 5 2 9 | 1 4 7 |
| 5 2 7 | 1 4 6 | 8 3 9 |
| 4 1 9 | 3 7 8 | 2 5 6 |
+-------+-------+-------+

